<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-18 09:52:34
-------------------
qualified stocks: 88
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.60 K
Current:  1.45 C
-------------------
Today PnL: -77.53 K (-0.53%)
Current PnL: -20.88 L (-13.68%)
CY Booked + Current PnL: -6.92 L (-4.53%)
-------------------
Total profit:  1.43 L
Total loss:  -22.30 L
-------------------
Total Booked + Current PnL: 20.06 L (15.97%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.63%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.37 C
Est FTT PnL: 91.70 L (63.26%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.26%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,150.00,85.0,M-SC,0.64,163841.0,-10718.0,10748.0,1.73,-6.14,6.56,0.02,234.0,-1.00,1.13,42.51,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,100.85,50.0,M-SC,3.61,86578.0,-14199.0,14294.0,-0.52,-14.09,16.51,0.09,253.0,-0.99,0.60,13.74,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-14.93,49.0,X-MC,3.67,88920.0,-1758.0,16664.0,0.27,-1.94,18.74,16.44,63.0,-0.11,0.61,17.85,X40,ATH,PHARMA
35,ICICIGI,2252.93,-15.31,55.0,X-MC,7.53,188260.0,13940.0,21255.0,-0.91,8.00,11.29,20.19,68.0,0.66,1.30,24.32,X40,ATH,INSURANCE
43,ITC,452.00,-40.74,44.0,X-LC,0.90,198167.0,-1971.0,22868.0,-0.45,-0.98,11.54,10.44,5.0,-0.09,1.37,4.52,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,116.35,71.0,H-SC,16.24,147056.0,-71440.0,99630.0,12.15,-32.70,67.75,12.90,161.0,-0.72,1.01,22.28,OX40N,NTT,FINANCE
39,INDIGOPNTS,1408.00,150.00,85.0,M-SC,0.64,163841.0,-10718.0,10748.0,1.73,-6.14,6.56,0.02,234.0,-1.00,1.13,42.51,OX40N,NTT,PAINTS
16,CERA,9475.00,-28.40,32.0,H-SC,2.84,130709.0,-45194.0,87222.0,1.11,-25.69,66.73,23.89,157.0,-0.52,0.90,12.88,OX40N,NTT,CERAMICS
64,SATIN,274.00,-23.78,60.0,H-SC,2.41,236540.0,-39744.0,182964.0,1.05,-14.39,77.35,51.83,146.0,-0.22,1.63,15.23,XY24,NTT,FINANCE
29,HAPPSTMNDS,1480.71,-25.05,46.0,H-SC,14.58,80808.0,-47357.0,156105.0,0.57,-36.95,193.18,84.85,147.0,-0.30,0.56,3.71,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SURYODAY,240.0,57.89,47.0,H-SC,13.23,143973.0,-35098.0,100824.0,-2.32,-19.60,70.03,36.71,167.0,-0.35,0.99,42.85,XR,NTT,BANKS
14,CAMPUS,393.0,-26.26,42.0,M-SC,3.46,153573.0,-14202.0,70045.0,-2.09,-8.47,45.61,33.28,222.0,-0.20,1.06,19.03,XY24,NTT,FOOTWEAR
56,QUESS,986.0,-53.17,33.0,X-SC,37.18,46669.0,-18337.0,169264.0,-2.09,-28.21,362.69,232.18,83.0,-0.11,0.32,0.59,XY24,NTT,MISC
13,BSOFT,831.7,-10.71,57.0,H-SC,8.30,97742.0,-42911.0,114339.0,-1.93,-30.51,116.98,50.79,151.0,-0.38,0.67,13.80,XR,ATH,IT
20,DABUR,735.0,-7.59,53.0,X-MC,2.67,249945.0,1877.0,106527.0,-1.70,0.76,42.62,43.70,72.0,0.02,1.72,15.92,XY24,BTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.93,46.0,M-SC,10.11,146898.0,-746.0,160075.0,-0.96,-0.51,108.97,107.91,216.0,-0.00,1.01,2.48,OX40N,ATH,CABLES
70,STARHEALTH,761.00,36.29,67.0,H-SC,15.65,280922.0,-201.0,133073.0,-0.74,-0.07,47.37,47.26,171.0,-0.00,1.94,50.51,XY24,NTT,INSURANCE
86,WIPRO,420.00,-14.70,48.0,M-LC,5.57,149866.0,-1079.0,110526.0,-0.95,-0.71,73.75,72.51,99.0,-0.01,1.03,5.26,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.93,46.0,M-SC,10.11,146898.0,-746.0,160075.0,-0.96,-0.51,108.97,107.91,216.0,-0.00,1.01,2.48,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,150.00,85.0,M-SC,0.64,163841.0,-10718.0,10748.0,1.73,-6.14,6.56,0.02,234.0,-1.00,1.13,42.51,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,100.85,50.0,M-SC,3.61,86578.0,-14199.0,14294.0,-0.52,-14.09,16.51,0.09,253.0,-0.99,0.60,13.74,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-66.17,47.0,H-SC,2.23,222300.0,-47367.0,83696.0,-0.28,-17.56,37.65,13.47,158.0,-0.57,1.53,13.03,XY24,NTT,PAINTS
68,SIS,528.00,2019.17,47.0,H-SC,2.38,85514.0,-25518.0,49128.0,0.04,-22.98,57.45,21.26,166.0,-0.52,0.59,15.54,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,22.90,61.0,M-SC,36.49,165231.0,16011.0,128170.0,-0.04,10.73,77.57,96.62,245.0,0.12,1.14,78.82,XY25,NTT,FINANCE
38,INDIAMART,4810.62,-47.77,50.0,H-SC,4.59,129914.0,6578.0,125055.0,-0.98,5.33,96.26,106.72,139.0,0.05,0.90,29.96,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.77,50.0,H-SC,4.59,129914.0,6578.0,125055.0,-0.98,5.33,96.26,106.72,139.0,0.05,0.90,29.96,AR,ATH,MISC
25,FINCABLES,1641.55,-18.93,46.0,M-SC,10.11,146898.0,-746.0,160075.0,-0.96,-0.51,108.97,107.91,216.0,-0.00,1.01,2.48,OX40N,ATH,CABLES
86,WIPRO,420.00,-14.70,48.0,M-LC,5.57,149866.0,-1079.0,110526.0,-0.95,-0.71,73.75,72.51,99.0,-0.01,1.03,5.26,XR,NTT,IT
85,WHIRLPOOL,2270.00,-42.48,37.0,M-SC,8.51,112728.0,-2070.0,96112.0,-0.83,-1.80,85.26,81.92,236.0,-0.02,0.78,33.45,XR,NTT,DURABLES
87,ZYDUSLIFE,1286.17,-15.85,33.0,H-MC,5.41,205375.0,-4199.0,78864.0,-0.58,-2.00,38.40,35.63,119.0,-0.05,1.42,14.33,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,9.46,23.0,X-SC,10.36,83498.0,-45172.0,90470.0,-0.85,-35.11,108.35,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
77,TMPV,1065.00,-20.63,25.0,X-LC,0.22,243922.0,-32488.0,453646.0,-0.08,-11.75,185.98,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
23,DMART,5391.45,-17.79,29.0,X-LC,5.23,200495.0,-9912.0,69071.0,-0.65,-4.71,34.45,28.12,19.0,-0.14,1.38,17.80,X40N,ATH,FMCG
34,HONAUT,58357.33,-26.21,30.0,X-SC,1.13,104955.0,-22983.0,70120.0,-1.31,-17.96,66.81,36.84,90.0,-0.33,0.72,7.45,X40N,ATH,ELECTRICAL
27,GLAXO,3466.20,-19.47,31.0,X-MC,6.91,199752.0,468.0,77544.0,0.15,0.23,38.82,39.15,60.0,0.01,1.38,29.30,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-20.63,25.0,X-LC,0.22,243922.0,-32488.0,453646.0,-0.08,-11.75,185.98,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
43,ITC,452.00,-40.74,44.0,X-LC,0.90,198167.0,-1971.0,22868.0,-0.45,-0.98,11.54,10.44,5.0,-0.09,1.37,4.52,X40,NTT,FMCG
34,HONAUT,58357.33,-26.21,30.0,X-SC,1.13,104955.0,-22983.0,70120.0,-1.31,-17.96,66.81,36.84,90.0,-0.33,0.72,7.45,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-21.15,59.0,X-MC,1.31,182349.0,5611.0,49125.0,-1.31,3.17,26.94,30.97,75.0,0.11,1.26,16.13,X40,ATH,INSURANCE
67,SIEMENS,4671.50,-0.55,63.0,H-LC,1.67,161890.0,-24205.0,71685.0,0.16,-13.01,44.28,25.51,49.0,-0.34,1.12,19.30,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,9.46,23.0,X-SC,10.36,83498.0,-45172.0,90470.0,-0.85,-35.11,108.35,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51769.93,-29.81,33.0,X-MC,7.95,156120.0,-7620.0,50958.0,-1.11,-4.65,32.64,26.47,55.0,-0.15,1.08,0.00,X40,ATH,APPARELS
77,TMPV,1065.00,-20.63,25.0,X-LC,0.22,243922.0,-32488.0,453646.0,-0.08,-11.75,185.98,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
56,QUESS,986.00,-53.17,33.0,X-SC,37.18,46669.0,-18337.0,169264.0,-2.09,-28.21,362.69,232.18,83.0,-0.11,0.32,0.59,XY24,NTT,MISC
9,BAJAJHFL,181.50,-19.01,33.0,X-MC,6.72,175086.0,-25420.0,122578.0,-0.59,-12.68,70.01,48.45,64.0,-0.21,1.21,0.96,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,986.00,-53.17,33.0,X-SC,37.18,46669.0,-18337.0,169264.0,-2.09,-28.21,362.69,232.18,83.0,-0.11,0.32,0.59,XY24,NTT,MISC
60,RELAXO,1176.00,-41.15,41.0,X-SC,3.01,80295.0,-64865.0,140797.0,-0.19,-44.69,175.35,52.31,91.0,-0.46,0.55,7.48,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,9.46,23.0,X-SC,10.36,83498.0,-45172.0,90470.0,-0.85,-35.11,108.35,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-14.93,49.0,X-MC,3.67,88920.0,-1758.0,16664.0,0.27,-1.94,18.74,16.44,63.0,-0.11,0.61,17.85,X40,ATH,PHARMA
34,HONAUT,58357.33,-26.21,30.0,X-SC,1.13,104955.0,-22983.0,70120.0,-1.31,-17.96,66.81,36.84,90.0,-0.33,0.72,7.45,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-27.18,55.0,X-LC,8.35,289614.0,-56342.0,123028.0,-0.68,-16.29,42.48,19.28,1.0,-0.46,2.00,7.07,X40,ATH,IT
41,INFY,2275.00,-21.03,50.0,X-LC,3.85,317711.0,4823.0,166862.0,-1.06,1.54,52.52,54.87,2.0,0.03,2.19,9.95,X40,BTT,IT
77,TMPV,1065.00,-20.63,25.0,X-LC,0.22,243922.0,-32488.0,453646.0,-0.08,-11.75,185.98,152.37,3.0,-0.07,1.68,0.00,XY24,NTT,AUTO
83,VBL,671.64,-18.89,46.0,X-LC,6.04,290928.0,-24914.0,137580.0,-1.23,-7.89,47.29,35.67,4.0,-0.18,2.01,4.87,X40N,ATH,FMCG
43,ITC,452.00,-40.74,44.0,X-LC,0.90,198167.0,-1971.0,22868.0,-0.45,-0.98,11.54,10.44,5.0,-0.09,1.37,4.52,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7722.22,67.0,L-SC,17.72,85836.0,-7974.0,84591.0,0.10,-8.50,98.55,81.67,271.0,-0.09,0.59,66.23,XR,NTT,CERAMICS
51,MASFIN,398.61,-18.70,51.0,H-SC,7.11,92475.0,-5505.0,27104.0,-1.38,-5.62,29.31,22.05,164.0,-0.20,0.64,34.42,XR,ATH,FINANCE
71,SURYODAY,240.00,57.89,47.0,H-SC,13.23,143973.0,-35098.0,100824.0,-2.32,-19.60,70.03,36.71,167.0,-0.35,0.99,42.85,XR,NTT,BANKS
81,VAIBHAVGBL,521.00,70.25,57.0,H-SC,10.28,147486.0,-35289.0,146365.0,-1.49,-19.31,99.24,60.77,159.0,-0.24,1.02,32.82,XR,NTT,JEWELLERY
66,SHALBY,327.00,1038.76,35.0,M-SC,17.01,154788.0,-26355.0,71822.0,0.06,-14.55,46.40,25.10,250.0,-0.37,1.07,24.05,XY24,NTT,HEALTHCARE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,150.00,85.0,M-SC,0.64,163841.0,-10718.0,10748.0,1.73,-6.14,6.56,0.02,234.0,-1.00,1.13,42.51,OX40N,NTT,PAINTS
81,VAIBHAVGBL,521.00,70.25,57.0,H-SC,10.28,147486.0,-35289.0,146365.0,-1.49,-19.31,99.24,60.77,159.0,-0.24,1.02,32.82,XR,NTT,JEWELLERY
5,ASIANPAINT,3460.25,0.21,83.0,X-LC,10.14,254769.0,3001.0,49731.0,0.41,1.19,19.52,20.95,31.0,0.06,1.76,37.25,X40,ATH,PAINTS
13,BSOFT,831.70,-10.71,57.0,H-SC,8.30,97742.0,-42911.0,114339.0,-1.93,-30.51,116.98,50.79,151.0,-0.38,0.67,13.80,XR,ATH,IT
0,5PAISA,593.00,116.35,71.0,H-SC,16.24,147056.0,-71440.0,99630.0,12.15,-32.70,67.75,12.90,161.0,-0.72,1.01,22.28,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.50
1,25,44.22
2,50,75.03


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.86
MC    28.56
LC    24.56
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.98
X40      20.46
X40N     11.52
XY25     10.08
XR        9.52
AR        9.05
OX40N     7.75
X200      1.80
SR        1.01
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.43
X-MC    22.20
X-LC    19.63
M-SC    13.28
X-SC     7.73
H-MC     4.67
H-LC     2.92
L-SC     1.42
M-MC     1.36
M-LC     1.03
L-LC     0.98
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.55,-4.88,38.80
IT,13.29,-15.52,75.20
FINANCE,10.61,-10.49,62.45
MISC,7.45,-20.35,81.06
PAINTS,6.04,-5.51,21.18
ELECTRICAL,5.63,-10.78,50.06
INSURANCE,4.50,2.97,31.23
PHARMA,3.41,-1.11,35.03
AUTO,3.35,-18.06,131.25


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3385707.0,21
XR,1273050.0,13
AR,1262242.0,10
X40,1005506.0,14
X40N,701029.0,9
OX40N,667415.0,10
XY25,470857.0,7
SR,274929.0,2
MH,66706.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3462833.0,25
M-SC,1425831.0,16
X-LC,1220924.0,11
X-MC,1177122.0,16
X-SC,842739.0,8
H-MC,402083.0,3
L-SC,255075.0,3
H-LC,134686.0,2
M-LC,110526.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1169420.0      6
           AR         846958.0      5
           XR         795150.0      7
M-SC       XY24       756108.0      6
X-LC       XY24       521083.0      2
X-MC       X40        417648.0      7
X-LC       X40        414405.0      5
X-SC       XY24       403586.0      3
X-MC       XY24       351291.0      3
H-SC       OX40N      309670.0      4
           SR         274929.0      2
M-SC       OX40N      271048.0      5
X-SC       X40N       265700.0      3
X-MC       X40N       228678.0      4
H-MC       AR         217864.0      2
X-LC       X40N       206651.0      2
H-MC       XY24       184219.0      1
X-MC       XY25       179505.0      2
X-SC       X40        173453.0      2
L-SC       XR         168378.0      2
M-SC       XR         144770.0      2
           XY25       128170.0      1
           AR         125735.0      2
M-LC       XR         110526.0      1
L-SC       OX40N       86697.0      1
X-LC       XY25        78785.0      2
H-LC       AR          71685.0      1
H-SC       MH          66706.0      1
H-LC       X200        63001.0      1
L-MC       XR          54226.0      1
M-MC       XY25        47730.0      1
L-LC       XY25        36667.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
